In [1]:
!pip install albumentations warmup_scheduler scikit-learn pandas matplotlib timm rasterio -q

## Imports

In [2]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, log_loss
import pickle
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler
import warnings
import sys
import pandas as pd
import os
import gc
import sys
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter
import cv2

import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from functools import partial

import argparse
import importlib
import torch
import torch.nn as nn
from torch.optim import AdamW
import torch.nn.functional as F

import datetime
import timm
import rasterio

## CFG

In [3]:
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2

class CFG:
    # ============== comp exp name =============
    comp_name = 'solafune'

    # comp_dir_path = './'
    comp_dir_path = 'input'
    comp_folder_name = ''
    comp_dataset_path = f'{comp_dir_path}{comp_folder_name}/'

    exp_name = 'exp1'

    # ============== pred target =============
    target_size = 1
    targets = 'class'

    # ============== model cfg =============
    model_name = 'xcit'
    backbone = 'seresnet50'
    num_classes = 1

    in_chans = 3
    # ============== training cfg =============
    img_size = 224

    train_batch_size = 8
    valid_batch_size = 2
    use_amp = True

    scheduler = 'GradualWarmupSchedulerV2'
    epochs = 30 # 30

    # adamW warmup
    warmup_factor = 10
    lr = 3e-4 / warmup_factor

    # ============== fold =============
    metric_direction = 'maximize'  # maximize, 'minimize'

    # ============== fixed =============
    pretrained = True
    inf_weight = 'best'  # 'best'

    min_lr = 1e-6
    weight_decay = 1e-6
    max_grad_norm = 1000
    num_workers = os.cpu_count()

    seed = 42

    # ============== set dataset path =============
    print('set dataset path')

    outputs_path = f'outputs/{comp_name}/{exp_name}/'

    submission_dir = outputs_path + 'submissions/'
    submission_path = submission_dir + f'submission_{exp_name}.csv'

    model_dir = outputs_path + \
        f'{comp_name}-models/'

    figures_dir = outputs_path + 'figures/'

    log_dir = outputs_path + 'logs/'
    log_path = log_dir + f'{exp_name}.txt'

    # ============== augmentation =============
    train_aug_list = [

        #A.Resize(img_size, img_size),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Transpose(p=0.5),
        A.RandomBrightnessContrast(p=0.3),
        ToTensorV2(),
    ]

    valid_aug_list = [
        #A.Resize(img_size, img_size),
        ToTensorV2(),
    ]
    TTA=True

set dataset path


## Utils

In [4]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [5]:
def init_logger(log_file):
    from logging import getLogger, INFO, StreamHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler = StreamHandler()
    handler.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler)
    return logger


def set_seed(seed=None, cudnn_deterministic=True):
    if seed is None:
        seed = 42

    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = cudnn_deterministic
    torch.backends.cudnn.benchmark = False

In [6]:
def make_dirs(cfg):
    for dir in [cfg.model_dir, cfg.figures_dir, cfg.submission_dir, cfg.log_dir]:
        os.makedirs(dir, exist_ok=True)

In [7]:
def cfg_init(cfg, mode='train'):
    set_seed(cfg.seed)
    # set_env_name()
    # set_dataset_path(cfg)

    if mode == 'train':
        make_dirs(cfg)

In [8]:
cfg_init(CFG)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Logger = init_logger(log_file=CFG.log_path)

Logger.info('\n\n-------- exp_info -----------------')
# Logger.info(datetime.datetime.now().strftime('%Y年%m月%d日 %H:%M:%S'))



-------- exp_info -----------------


## FE

In [9]:
def feature_engineering(image):
    #(2,10),(2,5), (3,8),(4,7),(1,9)
    l1 = (image[2, ...] - image[10, ...]) / (image[2, ...] + image[10, ...] + 1e-10)

    # NDBI: Normalized Difference Built-up Index, useful for identifying built-up areas.
    l2 = (image[2, ...] - image[5, ...]) / (image[2, ...] + image[5, ...] + 1e-10)
    
    # NDWI: Normalized Difference Water Index
    l3 = (image[4, ...] - image[7, ...]) / (image[4, ...] + image[7, ...] + 1e-10)
    l4 = (image[3, ...] - image[8, ...]) / (image[3, ...] + image[8, ...] + 1e-10)
    l5 = (image[1, ...] - image[9, ...]) / (image[1, ...] + image[9, ...] + 1e-10)

    # Stack the indices along with the original image channels
    image = np.concatenate(
        [
            image*2-1,
            np.expand_dims(l1, axis=0),
            np.expand_dims(l2, axis=0),
            np.expand_dims(l3, axis=0),
            np.expand_dims(l4, axis=0),
            np.expand_dims(l5, axis=0)
        ],
        axis=0,
    )
    return image


## Dataset

In [10]:
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as T

def apply_transforms(transform, image):
    return transform(image=image)['image']

class ImageDataset(Dataset):
    def __init__(self, df, root_dir, transform=None, is_train=True):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.annotations = df
        self.root_dir = root_dir
        self.transform = transform
        self.is_train = is_train

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.annotations.iloc[idx, 0])
        image = rasterio.open(img_name).read()
        image = feature_engineering(image).transpose(1,2,0)
        
        if self.is_train:
            target = self.annotations.iloc[idx, 1]
            target = torch.tensor(target, dtype=torch.float)

            if self.transform:
                image = apply_transforms(self.transform, image)
            return image, target
        else:

            if self.transform:
                image = apply_transforms(self.transform, image)
            return image, self.annotations.iloc[idx, 0]

## Model

In [11]:
"""import torch
import torch.nn.functional as F
from torch import nn
import timm
from torch.nn.parameter import Parameter"""

class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        # Load the pretrained ResNet50 model
        self.model = timm.create_model('maxvit_tiny_tf_512', pretrained=True, in_chans=17, num_classes=CFG.num_classes).to(device)
        #self.model = timm.create_model('maxvit_tiny_tf_224.in1k', pretrained=True, in_chans=17, num_classes=1).to(device)
        

    def forward(self, x):
        return self.model(x)



## Secheduler-Optimizer

In [12]:
"""import torch.nn as nn
import torch
import math
import time
import numpy as np
import torch"""

from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
from warmup_scheduler import GradualWarmupScheduler


class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    """
    https://www.kaggle.com/code/underwearfitting/single-fold-training-of-resnet200d-lb0-965
    """
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(
            optimizer, multiplier, total_epoch, after_scheduler)

    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [
                        base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]
        
def get_scheduler(cfg, optimizer):
    scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, cfg.epochs, eta_min=1e-7)
    scheduler = GradualWarmupSchedulerV2(
        optimizer, multiplier=10, total_epoch=1, after_scheduler=scheduler_cosine)

    return scheduler

def scheduler_step(scheduler, avg_val_loss, epoch):
    scheduler.step(epoch)

## Engine

In [13]:
from torch.nn import CrossEntropyLoss, MSELoss, BCEWithLogitsLoss

def criterion(y_pred, y_true):
     return BCEWithLogitsLoss()(y_pred, y_true)

In [14]:
device

device(type='cuda')

In [15]:
from sklearn.metrics import log_loss

def train_fn(train_loader, model, criterion, optimizer, device):
    model.train()

    scaler = GradScaler(enabled=CFG.use_amp)
    losses = AverageMeter()

    for step, (images, labels) in tqdm(enumerate(train_loader), total=len(train_loader)):
        images = images.to(device)
        labels = labels.to(device)
        
        batch_size = labels.size(0)

        with autocast(CFG.use_amp):
            y_preds = model(images)
            loss = criterion(y_preds.view(labels.shape), labels)

        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        #scaler.unscale_(optimizer)
        #grad_norm = torch.nn.utils.clip_grad_norm_(
        #    model.parameters(), CFG.max_grad_norm)

        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

    return losses.avg

def valid_fn(valid_loader, model, criterion, device):

    model.eval()
    losses = AverageMeter()

    val_outputs = []
    val_labels = []

    for step, (images, labels) in tqdm(enumerate(valid_loader), total=len(valid_loader)):
        images = images.to(device)
        labels = labels.to(device)

        batch_size = labels.size(0)

        with autocast(CFG.use_amp):
            y_preds = model(images)
            loss = criterion(y_preds.view(labels.shape), labels)
        losses.update(loss.item(), batch_size)

        y_preds = y_preds.sigmoid().view(labels.shape).cpu().detach().numpy()
        predictions = np.round(y_preds)
        val_outputs.append(predictions)
        val_labels.append(labels.cpu().detach().numpy())


    all_preds = np.concatenate(val_outputs)
    all_labels = np.concatenate(val_labels)
    

    f1 = f1_score(all_labels, all_preds)  # 'macro' averages the F1 score of each class
    #print('Calculating CV')
    #score = get_score(all_preds, all_labels)
    avg_loss = losses.avg

    return avg_loss, f1, all_preds, all_labels

def get_score(preds, labels):
    scores = []
    for i in range(len(CFG.targets)):
        score = log_loss(labels[:, i], preds[:, i])
        scores.append(score)
    score = np.mean(scores)
    Logger.info(f'score: {score}')
    return score

In [16]:
import pandas as pd
import os
import requests

train_df = pd.read_csv('/kaggle/input/mining/train/answer.csv')
train_df.loc[len(train_df)] = ['train_0.tif', 0]
train_df.columns = ['filename', 'class']

train_df.to_csv('train.csv', index=False)

#test
images_path = '/kaggle/input/mining/evaluation_images'

filenames = []

for file in os.listdir(images_path):
    filenames.append(file)
test_df = pd.DataFrame(filenames, columns=['filename'])

test_df.to_csv('test.csv', index=False)

# URL of the samplesubmission file
url = "https://solafune-dev-v1.s3-accelerate.amazonaws.com/competitions/mining_comp/uploadsample.csv"
response = requests.get(url)
with open('uploadsample.csv', 'wb') as file:
    file.write(response.content)
ss = pd.read_csv("/kaggle/working/uploadsample.csv")

In [17]:
from sklearn.model_selection import KFold, StratifiedKFold
folds = StratifiedKFold(5,shuffle=True,random_state = CFG.seed)
train_df['fold'] = -1
for idx,(train_idx,test_idx) in enumerate(folds.split(train_df, train_df[CFG.targets])) :
    train_df.loc[test_idx,'fold'] = idx


In [18]:
train_df.head()

,filename,class,fold
0,train_1.tif,0,3
1,train_2.tif,0,2
2,train_3.tif,1,2
3,train_4.tif,0,3
4,train_5.tif,0,2


In [19]:
def get_transforms(data, cfg):
    if data == 'train':
        aug = A.Compose(cfg.train_aug_list,is_check_shapes=False)
    elif data == 'valid':
        aug = A.Compose(cfg.valid_aug_list,is_check_shapes=False)

    # print(aug)
    return aug
def run() :

    oof_preds = []
    oof_labels = []
    for fold in range(5) :
        
        with torch.no_grad():
            torch.cuda.empty_cache()

        Logger.info(f'--------------Fold {fold+1}/5---------------------')
        df_train = train_df[train_df.fold!=fold].reset_index(drop=True)
        valid_indices = train_df[train_df.fold==fold].index
        df_valid = train_df[train_df.fold==fold].reset_index(drop=True)

        train_dataset = ImageDataset(df_train,root_dir='/kaggle/input/mining/train/train', is_train=True,transform = get_transforms('train', CFG))
        valid_dataset = ImageDataset(df_valid,root_dir='/kaggle/input/mining/train/train', is_train=True,transform = get_transforms('valid', CFG))

        train_loader = DataLoader(train_dataset,
                                batch_size=CFG.train_batch_size,
                                shuffle=True,
                                num_workers=CFG.num_workers, pin_memory=True, drop_last=True,
                                )
        valid_loader = DataLoader(valid_dataset,
                        batch_size=CFG.valid_batch_size,
                        shuffle=False,
                        num_workers=CFG.num_workers, pin_memory=True, drop_last=False
                                )
        model = CustomModel()
        model.to(device)

        optimizer = AdamW(model.parameters(), lr=CFG.lr)
        scheduler = get_scheduler(CFG, optimizer)
        best_score = - np.inf

        print('Start Training')
        for epoch in range(CFG.epochs):

                start_time = time.time()

                # train
                avg_loss = train_fn(train_loader, model, criterion, optimizer, device)

                # eval

                avg_val_loss, score, preds, labels = valid_fn(valid_loader, model, criterion, device)

                scheduler_step(scheduler, avg_val_loss, epoch)


                elapsed = time.time() - start_time

                Logger.info(
                        f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f} time: {elapsed:.0f}s')
                # Logger.info(f'Epoch {epoch+1} - avgScore: {avg_score:.4f}')
                Logger.info(
                        f'Epoch {epoch+1} - F1 score: {score:.4f}')

                if CFG.metric_direction == 'minimize':
                    update_best = score < best_score
                elif CFG.metric_direction == 'maximize':
                    update_best = score > best_score

                if update_best:
                    best_loss = avg_val_loss
                    best_score = score
                    best_preds = preds

                    Logger.info(
                        f'Epoch {epoch+1} - Save Best F1_Score: {best_score:.4f} Model')
                    Logger.info(
                        f'Epoch {epoch+1} - Save Best Loss: {best_loss:.4f} Model')

                    torch.save({'model': model.state_dict()},
                            CFG.model_dir + f'{CFG.exp_name}_fold{fold}_best.pth')
        train_df.loc[valid_indices, 'pred_' + CFG.targets] = best_preds

    return train_df


In [ ]:
train = run()

--------------Fold 1/5---------------------


model.safetensors:   0%|          | 0.00/124M [00:00<?, ?B/s]

Start Training


  0%|          | 0/124 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

...

  0%|          | 0/124 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

Epoch 12 - avg_train_loss: 0.2149  avg_val_loss: 0.2236 time: 98s
Epoch 12 - F1 score: 0.7778


  0%|          | 0/124 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

## Inference

In [ ]:
class EnsembleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.ModuleList()
        for fold in range(5):
            _model = CustomModel()
            _model.to(device)

            model_path = f'/kaggle/working/outputs/solafune/exp1/solafune-models/exp1_fold{fold}_best.pth'
            state = torch.load(model_path)['model']
            _model.load_state_dict(state)
            _model.eval()

            self.model.append(_model)

    def __len__(self) :
        return len(self.model)


    def forward(self,x):
        output=[]
        for m in self.model:
            output.append(m(x).sigmoid())
        output=torch.stack(output,dim=0).mean(0)
        return output

In [ ]:
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
def run_inference() :
    
    valid_dataset = ImageDataset(test_df,root_dir='/kaggle/input/mining/evaluation_images', is_train=False,transform = get_transforms('valid', CFG))
    valid_loader = DataLoader(valid_dataset,
                        batch_size=CFG.valid_batch_size,
                        shuffle=False,
                        num_workers=CFG.num_workers, pin_memory=True, drop_last=False
                                )
    model = EnsembleModel().to(device)
    all_probabilities = []
    file_names = []
    target_ratio = 0.206119163

    model.eval()
    losses = AverageMeter()

    val_outputs = []
    val_labels = []

    for step, (data, batch_file_names) in tqdm(enumerate(valid_loader), total=len(valid_loader)):
        data = data.to(device)
        
        # Forward pass
        outputs = model(data)

        # Apply sigmoid to get probabilities
        probabilities = torch.sigmoid(outputs)

        # Collect probabilities
        all_probabilities.extend(probabilities.cpu().detach().numpy())

        # Collect file names
        file_names.extend(batch_file_names)
        
    # Convert list of probabilities to numpy array
    all_probabilities = np.array(all_probabilities).squeeze()

    # Initialize the best threshold and its score
    best_threshold = None
    best_score = float('inf')

    # Check for thresholds in the range 0.1 to 0.99 with a step of 0.005
    for threshold in np.arange(0.1, 0.99, 0.005):
        predicted_classes = (all_probabilities > threshold).astype(int)
        class_1_ratio = np.mean(predicted_classes)

        # Calculate how close the class 1 ratio is to the target ratio
        score = abs(class_1_ratio - target_ratio)

        if score < best_score:
            best_score = score
            best_threshold = threshold

    # Using the best threshold, generate the final predictions
    final_classes = (all_probabilities > best_threshold).astype(int)
    df = pd.DataFrame({
        'file_name': file_names,
        'class': final_classes
    })
    
    # Set the first row as column names
    df.columns = df.iloc[0]

    # Drop the first row
    df = df.drop(df.index[0])

    # Reset the index
    df = df.reset_index(drop=True)

    return df, best_threshold
    



In [ ]:
predictions, optimal_threshold  = run_inference()
print('optimal threshold: ', optimal_threshold)

In [ ]:
predictions.iloc[:,1].value_counts(normalize=True)

In [ ]:
predictions.to_csv('submission.csv', index=False)